# Implementation of HOG extractor

- In this notebook, you will work on implementing HOG extractor. There are several **blanks** you need to fill in.
- Extraction of HOG feature from an image proceeds as follows:
    1. computing the gradient image in x and y
    2. computing the magnitude and orientation from the gradients
    3. computing gradient histograms
    4. normalizing across blocks
    5. flattening into a feature vector

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, exposure
from skimage.color import rgb2gray
from scipy import pi
from scipy.ndimage import uniform_filter

## 1. Computing the gradient image in x and y

In [19]:
def compute_gradients(image):
    """
    Compute the gradient of the image vertically and horizontally.
    
    Parameters
    ----------
    image : (X, Y) Input image array. 
    
    Returns
    -------
    I_x, I_y: the gradient on x-axis and y-axis.
    """
    
    image = np.atleast_2d(image)
    
    if image.ndim > 2:
        raise ValueError("Currently only supports grey-level images")\
    
    if image.dtype.kind == 'u':
        # convert uint image to float
        # to avoid problems with subtracting unsigned numbers in np.diff()
        image = image.astype('float')
        
    # initialize the parameters.
    I_x = np.zeros(image.shape)
    I_y = np.zeros(image.shape)
    
    ########## START CODE HERE ##########
    diff_x = np.diff(image, n=1, axis=1)
    diff_y = np.diff(image, n=1, axis=0)
    I_x[:, 1:-1] = (diff_x[:, :-1] + diff_x[:, 1:]) / 2 # TODO
    I_y[1:-1, :] = (diff_y[:-1] + diff_y[1:]) / 2 # TODO
    ########## END CODE HERE ##########
    
    return I_x, I_y

In [24]:
# test 1
c
I_x, I_y = compute_gradients(image)
assert (I_x == np.array([[0, 1, 1, 0]]* 4)).all()
assert (I_y == np.array([[0, 0, 0, 0], [4, 4, 4, 4], [4, 4, 4, 4], [0, 0, 0, 0]])).all()

## 2. computing the magnitude and orientation from the gradients

- Now we have the gradients $I_x, I_y$. Let's calculate magnitude $m$ and orientation $\theta$ from the gradients.
- Note that magnitude $m$ and orientation $\theta$ are calculated as follows:
$$
m(x,y) = \sqrt{{I_x(x,y)}^2 + {I_y(x,y)}^2}\\
\theta(x,y) = \tan^{-1}\frac{I_y(x,y)}{I_x(x,y)} \times \frac{180}{\pi}
$$
- $\theta$ is expressed in **degrees**

In [34]:
def compute_magnitude_and_orientation(I_x, I_y):
    """
    Compute the magnitude and orientation from the gradients I_x, I_y.
    
    Parameters
    ----------
    I_x: the gradient on x-axis
    I_y: the gradient on y-axis
    
    Returns
    -------
    m: magnitude of the gradient
    orientation: orientation of the gradient
    """
    
    magnitude = np.sqrt(I_x**2 + I_y**2) # TODO
    orientation = np.arctan2(I_y, I_x) * (180 / pi) % 180 # TODO

    return magnitude, orientation

In [38]:
# test
image = np.arange(16).reshape(4, 4)
I_x, I_y = compute_gradients(image)
magnitude, orientation = compute_magnitude_and_orientation(I_x, I_y)
expected_magnitude = np.load('./arrays_for_testing/magnitude.npy')
expected_orientation = np.load('./arrays_for_testing/orientation.npy')
assert np.sum((magnitude - expected_magnitude)**2) < 1e-3
assert np.sum((orientation - expected_orientation)**2) < 1e-3

## Computing gradient histograms